In [1]:
#Step 1: import the necessary modules
from __future__ import print_function
from keras.layers import Dense, Activation
from keras.layers.recurrent import SimpleRNN
from keras.models import Sequential
from keras.utils.vis_utils import plot_model
import numpy as np

Using TensorFlow backend.


In [2]:
#Step 2: Some preliminary cleanup is needed 
#The file contains line breaks and non-ASCII characters 

#open the text file as binary
from tkinter import filedialog
from tkinter import *
 
root = Tk()
root.filename =  filedialog.askopenfilename(initialdir = "/",
                                   title = "Select file",
                                   filetypes = (("Text File","*.txt"),("All Files","*.*")))
print (root.filename)
fin = open(root.filename, "rb") 
root.destroy()



#empty list
lines = []

#for each line in the file
for line in fin: 
    #remove leading and trailing white spaces, then convert it to lower case 
    line = line.strip().lower() 
    
    #Remove non-ASCII characters
    line = line.decode("ascii", "ignore")
    
    #if line is empty, jump
    if(len(line) == 0):
        continue
    
    #add the current line to lines list
    lines.append(line)

#close the file
fin.close()

#Concatenates each element of an iterable (such as list, string and tuple) 
#to the string and returns the concatenated string.
text = " ".join(lines)

print(text)

C:/Users/3734379/Desktop/AAAI Task/Language Model RNN/alice_in_wonderland.txt
project gutenbergs alices adventures in wonderland, by lewis carroll this ebook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever.  you may copy it, give it away or re-use it under the terms of the project gutenberg license included with this ebook or online at www.gutenberg.org title: alices adventures in wonderland author: lewis carroll posting date: june 25, 2008 [ebook #11] release date: march, 1994 last updated: october 6, 2016 language: english character set encoding: utf-8 *** start of this project gutenberg ebook alices adventures in wonderland *** alices adventures in wonderland lewis carroll the millennium fulcrum edition 3.0 chapter i. down the rabbit-hole alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversati

In [3]:
#Step 3: create the lookup tables between characters and their indices 

#get the set of all unique characters in text
chars = set([c for c in text])
print(chars)

#the number of different unique characters
nb_chars = len(chars)
print(nb_chars)

#dictionary of character and its index
char2index = dict((c,i) for i,c in enumerate(chars))
print(char2index)

#dictionary of index and the character
index2char = dict((i,c) for i,c in enumerate(chars))
print(index2char)

{'m', 'k', ':', '0', '*', 'z', ',', '8', '6', ']', '4', '[', '-', '?', 'v', '9', 'w', 'l', 'i', ' ', 's', '5', '#', '/', '2', 'j', 'd', '(', 'x', '%', '$', 'e', 'y', '3', 'f', '!', 'h', 'n', '7', 't', ')', 'b', '@', 'u', 'c', 'r', 'o', '.', '1', 'p', '_', 'a', 'q', 'g', ';'}
55
{'m': 0, 'k': 1, ':': 2, '0': 3, '*': 4, 'z': 5, ',': 6, '8': 7, '6': 8, ']': 9, '4': 10, '[': 11, '-': 12, '?': 13, 'v': 14, '9': 15, 'w': 16, 'l': 17, 'i': 18, ' ': 19, 's': 20, '5': 21, '#': 22, '/': 23, '2': 24, 'j': 25, 'd': 26, '(': 27, 'x': 28, '%': 29, '$': 30, 'e': 31, 'y': 32, '3': 33, 'f': 34, '!': 35, 'h': 36, 'n': 37, '7': 38, 't': 39, ')': 40, 'b': 41, '@': 42, 'u': 43, 'c': 44, 'r': 45, 'o': 46, '.': 47, '1': 48, 'p': 49, '_': 50, 'a': 51, 'q': 52, 'g': 53, ';': 54}
{0: 'm', 1: 'k', 2: ':', 3: '0', 4: '*', 5: 'z', 6: ',', 7: '8', 8: '6', 9: ']', 10: '4', 11: '[', 12: '-', 13: '?', 14: 'v', 15: '9', 16: 'w', 17: 'l', 18: 'i', 19: ' ', 20: 's', 21: '5', 22: '#', 23: '/', 24: '2', 25: 'j', 26: 'd', 2

In [4]:
#Step 4: Create the input and its label
#Iterate over string text and get a span of characters (as input), and next character is the label

#span size
SEQLEN = 10

#step size
STEP = 1

input_chars = []
label_chars = []

#for each characters span of size SEQLEN, construct inputs and labels  
for i in range(0, len(text)-SEQLEN, STEP):
    input_chars.append(text[i: i+SEQLEN])
    #print(text[i: i+SEQLEN])
    
    label_chars.append(text[i+SEQLEN])
    #print(text[i+SEQLEN])

print(len(input_chars))
print(len(label_chars))

158773
158773


In [5]:
#Step 5: vectorize input and label texts

# For each input, there are SEQLEN (10) characters.
#There are nb_chars unique characters = vocabulary size = 55
#Each input character will be represented as one-hot encoded vector of size nb_chars
#each input characters vector is a tensor of size (SEQLEN * nb_chars)

#Tensor level 3, span size * char set size * size of training set <change for numpy>
X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)

#Tensor of level 2, char set size * size of training set <change for numpy>
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)

#for each characters span
for i, input_char in enumerate(input_chars): 
    #for each character in the span
    for j, ch in enumerate(input_char):
        
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1
    




In [6]:
#Step 6: Build the model 

#RNN output dimension = hidden size = 128

#a single character will be the output, return_sequence = False

#single input to RNN will have the shape (span size * char set size)
#all inputs to RNN will have the shape (span size * char set size * size of training set)

#unroll = True

#RNN is connected to a dense (fully connected) layer
#Its size = char set size
#It emits scores for each of the characters in the char set
#The activation of the dense layer is softmax (normalizes scores to prob.)
#The char of the hightest prob. is the prediction
#Loss function is categorical cross-entropy
#RMSprop optimizer

HIDDEN_SIZE = 128
BATCH_SIZE = 128
NUM_ITERATIONS = 25
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100

model = Sequential()
model.add(SimpleRNN(HIDDEN_SIZE, return_sequences=False, 
          input_shape = (SEQLEN, nb_chars),
          unroll = True ))
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [7]:
#We do not have label data
#We train for NUM_ITERATIONS EPOCHS, and test the model after each epoch
#NUM_EPOCHS_PER_ITERATION = 1
#each Iteration contains: certain number of training epochs + testing 

#for each epoch = for each iteration
for iteration in range(NUM_ITERATIONS):
    
    #printing the iteration number
    print("=" * 50)
    print("Iteration #: " + str(iteration))
    
    #Training the model with the dataset ################################################
    model.fit(X, y, batch_size = BATCH_SIZE, epochs = NUM_EPOCHS_PER_ITERATION)
    
    #Testing ############################################################################
    #generate random int number from 0 to len(input_chars)-1     
    test_idx = np.random.randint(len(input_chars)) 
    
    #pick randomly an element (10 characters) from input_chars
    test_chars = input_chars[test_idx]
    
    resultant_string = test_chars
    
    #for each prediction
    for i in range(NUM_PREDS_PER_EPOCH):
        
        #Xtest is one frame from X
        Xtest = np.zeros((1, SEQLEN, nb_chars))
        
        #for each character in test_chars  
        for j, ch in enumerate(test_chars):
            #hot encoded vector for each char
            Xtest[0, j, char2index[ch]] = 1 #0 for 3rd dimension
        
        #apply the Xtest to the trained model
        #the output is a vector contains the probabilities
        pred = model.predict(Xtest, verbose=0)[0]
        #print("The output vector is: ", end="")
        #print(pred)
        
        #get the index of the max. probability
        #get the corresponding char of that index
        ypred = index2char[np.argmax(pred)]
        print("The test chars: " + test_chars)
        print("The predicted char is: " + ypred)
        
        #remove the first char from test_chars & append to it ypred
        test_chars = test_chars[1:] + ypred
        
        resultant_string = resultant_string + ypred
    
    print("After iteration # " + str(iteration) + ": " + resultant_string)

Iteration #: 0
Epoch 1/1
158773/158773 [==============================] - 14s 90us/step - loss: 2.3413
The test chars: he person 
The predicted char is: t
The test chars: e person t
The predicted char is: h
The test chars:  person th
The predicted char is: e
The test chars: person the
The predicted char is:  
The test chars: erson the 
The predicted char is: h
The test chars: rson the h
The predicted char is: a
The test chars: son the ha
The predicted char is: n
The test chars: on the han
The predicted char is: d
The test chars: n the hand
The predicted char is:  
The test chars:  the hand 
The predicted char is: t
The test chars: the hand t
The predicted char is: h
The test chars: he hand th
The predicted char is: e
The test chars: e hand the
The predicted char is:  
The test chars:  hand the 
The predicted char is: t
The test chars: hand the t
The predicted char is: h
The test chars: and the th
The predicted char is: e
The test chars: nd the the
The predicted char is:  
The test char

The test chars: e coust of
The predicted char is:  
The test chars:  coust of 
The predicted char is: t
The test chars: coust of t
The predicted char is: h
The test chars: oust of th
The predicted char is: e
The test chars: ust of the
The predicted char is:  
The test chars: st of the 
The predicted char is: h
The test chars: t of the h
The predicted char is: e
The test chars:  of the he
The predicted char is: r
The test chars: of the her
The predicted char is: e
The test chars: f the here
The predicted char is: s
The test chars:  the heres
The predicted char is:  
The test chars: the heres 
The predicted char is: i
The test chars: he heres i
The predicted char is: n
The test chars: e heres in
The predicted char is:  
The test chars:  heres in 
The predicted char is: t
The test chars: heres in t
The predicted char is: h
The test chars: eres in th
The predicted char is: e
The test chars: res in the
The predicted char is:  
The test chars: es in the 
The predicted char is: c
The test cha

The test chars: u the mage
The predicted char is:  
The test chars:  the mage 
The predicted char is: o
The test chars: the mage o
The predicted char is: u
After iteration # 3: hon. i meat out of the mouth ure mound the mack ou the mage out of the mouth ure mound the mack ou the mage ou
Iteration #: 4
Epoch 1/1
158773/158773 [==============================] - 11s 70us/step - loss: 1.7989
The test chars: the terms 
The predicted char is: t
The test chars: he terms t
The predicted char is: h
The test chars: e terms th
The predicted char is: e
The test chars:  terms the
The predicted char is:  
The test chars: terms the 
The predicted char is: r
The test chars: erms the r
The predicted char is: a
The test chars: rms the ra
The predicted char is: b
The test chars: ms the rab
The predicted char is: b
The test chars: s the rabb
The predicted char is: e
The test chars:  the rabbe
The predicted char is: t
The test chars: the rabbet
The predicted char is:  
The test chars: he rabbet 
The predic

158773/158773 [==============================] - 12s 74us/step - loss: 1.6960
The test chars: l, and the
The predicted char is:  
The test chars: , and the 
The predicted char is: g
The test chars:  and the g
The predicted char is: o
The test chars: and the go
The predicted char is: t
The test chars: nd the got
The predicted char is:  
The test chars: d the got 
The predicted char is: o
The test chars:  the got o
The predicted char is: f
The test chars: the got of
The predicted char is:  
The test chars: he got of 
The predicted char is: t
The test chars: e got of t
The predicted char is: h
The test chars:  got of th
The predicted char is: e
The test chars: got of the
The predicted char is:  
The test chars: ot of the 
The predicted char is: s
The test chars: t of the s
The predicted char is: a
The test chars:  of the sa
The predicted char is: i
The test chars: of the sai
The predicted char is: d
The test chars: f the said
The predicted char is:  
The test chars:  the said 
The predict

158773/158773 [==============================] - 12s 73us/step - loss: 1.6237
The test chars: c domain p
The predicted char is: l
The test chars:  domain pl
The predicted char is: e
The test chars: domain ple
The predicted char is: a
The test chars: omain plea
The predicted char is: s
The test chars: main pleas
The predicted char is: e
The test chars: ain please
The predicted char is:  
The test chars: in please 
The predicted char is: t
The test chars: n please t
The predicted char is: o
The test chars:  please to
The predicted char is:  
The test chars: please to 
The predicted char is: h
The test chars: lease to h
The predicted char is: e
The test chars: ease to he
The predicted char is: r
The test chars: ase to her
The predicted char is:  
The test chars: se to her 
The predicted char is: a
The test chars: e to her a
The predicted char is: l
The test chars:  to her al
The predicted char is: l
The test chars: to her all
The predicted char is:  
The test chars: o her all 
The predict

...

The test chars: ittle the 
The predicted char is: p
The test chars: ttle the p
The predicted char is: r
The test chars: tle the pr
The predicted char is: o
The test chars: le the pro
The predicted char is: j
The test chars: e the proj
The predicted char is: e
The test chars:  the proje
The predicted char is: c
The test chars: the projec
The predicted char is: t
The test chars: he project
The predicted char is:  
The test chars: e project 
The predicted char is: g
The test chars:  project g
The predicted char is: u
The test chars: project gu
The predicted char is: t
The test chars: roject gut
The predicted char is: e
After iteration # 16: , turning to the project gutenberg-tm electronic works to say of the mock turtle was a little the project gute
Iteration #: 17
Epoch 1/1
158773/158773 [==============================] - 11s 70us/step - loss: 1.4577
The test chars:  i cant te
The predicted char is: l
The test chars: i cant tel
The predicted char is: l
The test chars:  cant tell
The pred

The test chars: ronic work
The predicted char is:  
The test chars: onic work 
The predicted char is: o
The test chars: nic work o
The predicted char is: f
The test chars: ic work of
The predicted char is:  
The test chars: c work of 
The predicted char is: t
The test chars:  work of t
The predicted char is: h
The test chars: work of th
The predicted char is: e
The test chars: ork of the
The predicted char is:  
The test chars: rk of the 
The predicted char is: p
The test chars: k of the p
The predicted char is: r
The test chars:  of the pr
The predicted char is: o
The test chars: of the pro
The predicted char is: j
The test chars: f the proj
The predicted char is: e
The test chars:  the proje
The predicted char is: c
The test chars: the projec
The predicted char is: t
The test chars: he project
The predicted char is:  
The test chars: e project 
The predicted char is: g
The test chars:  project g
The predicted char is: u
The test chars: project gu
The predicted char is: t
The test cha

The test chars:  the door 
The predicted char is: t
The test chars: the door t
The predicted char is: h
The test chars: he door th
The predicted char is: e
The test chars: e door the
The predicted char is:  
The test chars:  door the 
The predicted char is: d
The test chars: door the d
The predicted char is: o
The test chars: oor the do
The predicted char is: o
The test chars: or the doo
The predicted char is: r
The test chars: r the door
The predicted char is:  
The test chars:  the door 
The predicted char is: t
The test chars: the door t
The predicted char is: h
The test chars: he door th
The predicted char is: e
The test chars: e door the
The predicted char is:  
The test chars:  door the 
The predicted char is: d
The test chars: door the d
The predicted char is: o
The test chars: oor the do
The predicted char is: o
The test chars: or the doo
The predicted char is: r
The test chars: r the door
The predicted char is:  
The test chars:  the door 
The predicted char is: t
The test cha

The test chars: r and the 
The predicted char is: d
The test chars:  and the d
The predicted char is: o
The test chars: and the do
The predicted char is: o
The test chars: nd the doo
The predicted char is: r
The test chars: d the door
The predicted char is:  
The test chars:  the door 
The predicted char is: a
The test chars: the door a
The predicted char is: n
The test chars: he door an
The predicted char is: d
The test chars: e door and
The predicted char is:  
The test chars:  door and 
The predicted char is: t
The test chars: door and t
The predicted char is: h
The test chars: oor and th
The predicted char is: e
The test chars: or and the
The predicted char is:  
The test chars: r and the 
The predicted char is: d
The test chars:  and the d
The predicted char is: o
The test chars: and the do
The predicted char is: o
The test chars: nd the doo
The predicted char is: r
The test chars: d the door
The predicted char is:  
The test chars:  the door 
The predicted char is: a
The test cha

The test chars: anged to h
The predicted char is: e
The test chars: nged to he
The predicted char is: r
The test chars: ged to her
The predicted char is: s
The test chars: ed to hers
The predicted char is: e
The test chars: d to herse
The predicted char is: l
The test chars:  to hersel
The predicted char is: f
The test chars: to herself
The predicted char is: ,
The test chars: o herself,
The predicted char is:  
The test chars:  herself, 
The predicted char is: a
The test chars: herself, a
The predicted char is: n
The test chars: erself, an
The predicted char is: d
The test chars: rself, and
The predicted char is:  
The test chars: self, and 
The predicted char is: s
The test chars: elf, and s
The predicted char is: h
The test chars: lf, and sh
The predicted char is: e
The test chars: f, and she
The predicted char is:  
The test chars: , and she 
The predicted char is: w
The test chars:  and she w
The predicted char is: a
The test chars: and she wa
The predicted char is: s
The test cha